In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import os
import torch
from torchvision import transforms
from transformers import ViTFeatureExtractor, ViTForImageClassification
from PIL import Image
from torch.utils.data import DataLoader, Dataset

# Define the directory paths
train_path = "/kaggle/input/the-wildfire-dataset/the_wildfire_dataset/the_wildfire_dataset/train"
val_path = "/kaggle/input/the-wildfire-dataset/the_wildfire_dataset/the_wildfire_dataset/val"
test_path = "/kaggle/input/the-wildfire-dataset/the_wildfire_dataset/the_wildfire_dataset/test"

# ViT Feature Extractor
feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224-in21k')

# Define a custom dataset class
class WildfireDataset(Dataset):
    def __init__(self, folder_path, feature_extractor):
        self.folder_path = folder_path
        self.image_files = []  # List to store image file paths
        self.labels = []  # List to store corresponding labels
        label_mapping = {
            "Smoke_confounding_elements": 0,
            "Forested_areas_without_confounding_elements": 1,
            "Fire_confounding_elements": 2
#             "Smoke_from_fires": 3,
#             "Both_smoke_and_fire": 4,
        }


        # Populate image_files and labels based on the clarified folder structure
        for class_label in ["nofire", "fire"]:
            current_path = f"{folder_path}/{class_label}"
            if class_label == "nofire":
                for subclass_label in ["Forested_areas_without_confounding_elements",
                                       "Fire_confounding_elements",
                                       "Smoke_confounding_elements"]:
                    current_subclass_path = f"{current_path}/{subclass_label}"
                    image_files = os.listdir(current_subclass_path)
                    self.image_files.extend([f"{current_subclass_path}/{img}" for img in image_files])
                    self.labels.extend([label_mapping[subclass_label]] * len(image_files))
            else:
                continue

        self.feature_extractor = feature_extractor

    def __getitem__(self, idx):
        img_path = self.image_files[idx]
        label = self.labels[idx]
        image = Image.open(img_path).convert("RGB")
        
        # Ensure that the image has the correct shape (num_channels, height, width)
        image = self.feature_extractor(images=image, return_tensors="pt")["pixel_values"].squeeze(0)

        return {
            "pixel_values": image,
            "labels": torch.tensor(label),
        }

    
    def __len__(self):
        return len(self.image_files)

# Create dataset instances
train_dataset = WildfireDataset(train_path, feature_extractor)
# val_dataset = WildfireDataset(val_path, feature_extractor)
test_dataset = WildfireDataset(test_path, feature_extractor)

# Create DataLoader instances
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
# val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

2024-02-13 12:26:21.756794: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-13 12:26:21.756919: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-13 12:26:22.028474: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [ ]:
# Check the length of the subset dataset
print("Number of samples in the subset dataset:", len(train_dataset))

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs!")
#     model = torch.nn.DataParallel(model)
# model.to(device)

In [ ]:
import torch
from transformers import ViTForImageClassification
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score
import torch.nn as nn

# Load the fine-tuned model
model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224-in21k',num_labels=3)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
    
# Define optimizer and loss function
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
loss_fn = torch.nn.CrossEntropyLoss()

# Define training parameters
num_epochs = 5
accuracies = []

# Training loop
for epoch in range(num_epochs):
    model.train()
    train_losses = []
    epoch_labels = []
    epoch_predictions = []

    for batch in train_dataloader:
        inputs = batch["pixel_values"].to(device)
        labels = batch["labels"].to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        logits = outputs.logits
        loss = loss_fn(logits, labels)
        loss.backward()
        optimizer.step()

        train_losses.append(loss.item())
        # Store predictions and labels for accuracy calculation
        epoch_labels.extend(labels.cpu().numpy())
        epoch_predictions.extend(torch.argmax(outputs.logits, axis=1).cpu().numpy())
    
    # Calculate accuracy
    accuracy = accuracy_score(epoch_labels, epoch_predictions)
    accuracies.append(accuracy)

    # Print average training loss for the epoch
    avg_train_loss = sum(train_losses) / len(train_losses)
    print(f"Epoch {epoch + 1}/{num_epochs}, Avg Train Loss: {avg_train_loss:.4f}")

In [5]:
model = ViTForImageClassification.from_pretrained('/kaggle/input/capstone-models/nofire_subclass_model_VIT/nofire_subclass_model_VIT',num_labels=3)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

ViTForImageClassification(
  (vit): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_features=7

In [6]:
import torch
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score
import numpy as np

import torch.nn as nn

# Define the loss function (criterion)
criterion = nn.CrossEntropyLoss()
# Initialize val_accuracies list
val_accuracies = []

# Validation loop
model.eval()
val_losses = []
val_preds = []
val_labels = []

with torch.no_grad():
    for batch in test_dataloader:
        inputs = batch["pixel_values"].to(device)
        labels = batch["labels"].to(device)

        # Check for non-empty batches
        if inputs.size(0) == 0:
            continue

        outputs = model(inputs, labels=labels)
        
        # Check for NaN values in the loss
        if np.any(np.isnan(outputs.loss.cpu().numpy())):
            print("NaN loss encountered. Skipping batch.")
            continue

        # Compute the loss using the appropriate criterion
        loss = criterion(outputs.logits, labels)
        val_losses.append(loss.item())

        predictions = torch.argmax(outputs.logits, dim=1)
        val_preds.extend(predictions.cpu().numpy())
        val_labels.extend(labels.cpu().numpy())

# Calculate validation accuracy
val_accuracy = accuracy_score(val_labels, val_preds)
val_loss = sum(val_losses) / len(val_losses)

print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")
print(f"Validation Loss: {val_loss:.4f}")

# Store validation accuracy for further analysis
val_accuracies.append(val_accuracy)

/opt/conda/lib/python3.10/site-packages/PIL/Image.py:3176: DecompressionBombWarning: Image size (101859328 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/PIL/Image.py:3176: DecompressionBombWarning: Image size (96631920 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/PIL/Image.py:3176: DecompressionBombWarning: Image size (94487082 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


Validation Accuracy: 89.24%
Validation Loss: 0.3459


In [ ]:
# Save the fine-tuned model
if isinstance(model, torch.nn.DataParallel):
    model.module.save_pretrained("nofire_subclass_model_VIT")
else:
    model.save_pretrained("nofire_subclass_model_VIT")

In [ ]:
# Save the fine-tuned model
model.save_pretrained("nofire_subclass_model_VIT")

In [ ]:
# Create a zip archive of a folder
!zip -r nofire_subclass_model_VIT.zip nofire_subclass_model_VIT

**nofire_subclass_model_VIT_10epochs**

In [ ]:
import torch
from transformers import ViTForImageClassification
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score

# Load the fine-tuned model
model = ViTForImageClassification.from_pretrained("/kaggle/input/capstone-models/nofire_subclass_model_VIT/nofire_subclass_model_VIT")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define optimizer and loss function
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
loss_fn = torch.nn.CrossEntropyLoss()  # Cross-entropy loss handles labels starting from 0

# Define training parameters
num_epochs = 5

# for param in model.base_model.parameters():
#     param.requires_grad = False  # Freeze initial layers


# Training loop
start_epoch = 6  # Start from the next epoch after the previous training

for epoch in range(start_epoch, start_epoch + num_epochs):
    model.train()
    train_losses = []

    for batch in train_dataloader:
        inputs = batch["pixel_values"].to(device)
        labels = batch["labels"].to(device)

        optimizer.zero_grad()
        outputs = model(inputs, labels=labels)
        logits = outputs.logits
        loss = loss_fn(logits, labels)
        loss.backward()
        optimizer.step()

        train_losses.append(loss.item())

    # Print average training loss for the epoch4
    avg_train_loss = sum(train_losses) / len(train_losses)
    print(f"Epoch {epoch + 1}/{num_epochs}, Avg Train Loss: {avg_train_loss:.4f}")

In [7]:
model = ViTForImageClassification.from_pretrained('/kaggle/input/capstone-models/nofire_subclass_model_VIT_10epochs/nofire_subclass_model_VIT_10epochs',num_labels=3)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

ViTForImageClassification(
  (vit): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_features=7

In [8]:
import torch
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score
import numpy as np

import torch.nn as nn

# Define the loss function (criterion)
criterion = nn.CrossEntropyLoss()
# Initialize val_accuracies list
val_accuracies = []

# Validation loop
model.eval()
val_losses = []
val_preds = []
val_labels = []

with torch.no_grad():
    for batch in test_dataloader:
        inputs = batch["pixel_values"].to(device)
        labels = batch["labels"].to(device)

        # Check for non-empty batches
        if inputs.size(0) == 0:
            continue

        outputs = model(inputs, labels=labels)
        
        # Check for NaN values in the loss
        if np.any(np.isnan(outputs.loss.cpu().numpy())):
            print("NaN loss encountered. Skipping batch.")
            continue

        # Compute the loss using the appropriate criterion
        loss = criterion(outputs.logits, labels)
        val_losses.append(loss.item())

        predictions = torch.argmax(outputs.logits, dim=1)
        val_preds.extend(predictions.cpu().numpy())
        val_labels.extend(labels.cpu().numpy())

# Calculate validation accuracy
val_accuracy = accuracy_score(val_labels, val_preds)
val_loss = sum(val_losses) / len(val_losses)

print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")
print(f"Validation Loss: {val_loss:.4f}")

# Store validation accuracy for further analysis
val_accuracies.append(val_accuracy)

/opt/conda/lib/python3.10/site-packages/PIL/Image.py:3176: DecompressionBombWarning: Image size (101859328 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/PIL/Image.py:3176: DecompressionBombWarning: Image size (96631920 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/PIL/Image.py:3176: DecompressionBombWarning: Image size (94487082 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


Validation Accuracy: 92.43%
Validation Loss: 0.2538


In [ ]:
# Save the fine-tuned model
model.save_pretrained("nofire_subclass_model_VIT_10epochs")

In [ ]:
# Create a zip archive of a folder
!zip -r nofire_subclass_model_VIT_10epochs.zip nofire_subclass_model_VIT_10epochs